In [1]:
import os
import cv2
import numpy as np
import pickle
from mtcnn import MTCNN
from keras_facenet import FaceNet

# Initialize the face detector and embedder
detector = MTCNN()
embedder = FaceNet()

# Function to preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return rgb_image

# Function to detect faces in an image
def detect_faces(image):
    faces = detector.detect_faces(image)
    return faces

# Function to generate embeddings for a face image
def generate_embedding(image):
    embeddings = embedder.embeddings([image])
    return embeddings[0]

# Function to save embeddings to a folder
def save_embedding(name, embeddings, folder='embeddings'):
    if not os.path.exists(folder):
        os.makedirs(folder)
    filepath = os.path.join(folder, f'{name}.pkl')
    with open(filepath, 'wb') as f:
        pickle.dump(embeddings, f)

# Function to load embeddings from a folder
def load_embeddings(folder='embeddings'):
    embeddings = {}
    for filename in os.listdir(folder):
        if filename.endswith('.pkl'):
            name = os.path.splitext(filename)[0]
            filepath = os.path.join(folder, filename)
            with open(filepath, 'rb') as f:
                embedding_list = pickle.load(f)
            embeddings[name] = embedding_list
    return embeddings

# Function to find a match for a new embedding
def find_match(new_embedding, threshold=0.6):
    known_embeddings = load_embeddings()
    for name, embedding_list in known_embeddings.items():
        for embedding in embedding_list:
            distance = np.linalg.norm(new_embedding - embedding)
            if distance < threshold:
                return name
    return None

# Function to recognize faces in an image and handle new faces
def recognize_faces_in_image(image_path):
    image = preprocess_image(image_path)
    faces = detect_faces(image)
    for face in faces:
        x, y, width, height = face['box']
        face_image = image[y:y+height, x:x+width]
        embedding = generate_embedding(face_image)
        name = find_match(embedding)
        if name:
            print(f'Face recognized: {name}')
        else:
            print('Face not recognized')
            new_name = input('Enter name for the new face: ')
            person_folder = os.path.join('images', new_name)
            if not os.path.exists(person_folder):
                os.makedirs(person_folder)
            new_image_path = os.path.join(person_folder, f'1.jpg')
            cv2.imwrite(new_image_path, cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR))
            save_embedding(new_name, [embedding])


In [7]:
# Add known faces to the system
known_folders = [f for f in os.listdir('images') if os.path.isdir(os.path.join('images', f))]

for person in known_folders:
    embeddings = []
    person_folder = os.path.join('images', person)
    for image_name in os.listdir(person_folder):
        image_path = os.path.join(person_folder, image_name)
        image = preprocess_image(image_path)
        faces = detect_faces(image)
        for face in faces:
            x, y, width, height = face['box']
            face_image = image[y:y+height, x:x+width]
            embedding = generate_embedding(face_image)
            embeddings.append(embedding)
    save_embedding(person, embeddings)

# Recognize faces in a new image
recognize_faces_in_image('test.JPG')

1/1 [==============================] - 0s 42ms/step
Face recognized: sofia
1/1 [==============================] - 0s 37ms/step
Face not recognized
